<a href="https://colab.research.google.com/github/Shadman19/Parameterized-ML-model/blob/main/Parameterized_Machine_Learning_Model_Training_with_PyTorch_Shadman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction:
Overview and Instructions
The notebook will begin with a brief overview of the purpose, instructions on how to select datasets, models, and configure the training loop.


# Machine Learning Model Training with PyTorch
This notebook allows flexible, parameterized training of models using PyTorch. You can select datasets and models from the `torchvision` library, and configure training parameters like batch size, learning rate, and epochs.

## Features:
- Support for various datasets and models from `torchvision`.
- Robust error handling with suggestions for typos.
- Optional verbosity flag for detailed training output.
- Early stopping based on validation loss.



# 2. Setup: Import Libraries and Dependencies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np

# For error handling and suggestions
import difflib

# 3. Dataset Selection: Parameterized Dataset Loading with Error Handling
We need to create a flexible loading mechanism for the dataset with error handling for typos.

In [ ]:
# Function to list all available datasets
def list_available_datasets():
    return [name for name in dir(datasets) if not name.startswith('__')]

# Function to load dataset based on parameterized input
def load_dataset(dataset_name, batch_size):
    available_datasets = list_available_datasets()

    if dataset_name not in available_datasets:
        # Suggest closest matches for typos
        suggestions = difflib.get_close_matches(dataset_name, available_datasets)
        raise ValueError(f"Dataset '{dataset_name}' not found. Did you mean: {suggestions}?")

    # Default transformations
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

    # Dataset loading
    dataset_class = getattr(datasets, dataset_name)

    train_dataset = dataset_class(root='./data', train=True, download=True, transform=transform)
    test_dataset = dataset_class(root='./data', train=False, download=True, transform=transform)

    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


# 4. Model Selection: Parameterized Model Loading with Error Handling

In [ ]:
# Function to list all available models
def list_available_models():
    return [name for name in dir(models) if callable(getattr(models, name)) and not name.startswith('__')]

# Function to load model based on parameterized input
def load_model(model_name, num_classes=10):
    available_models = list_available_models()

    if model_name not in available_models:
        # Suggest closest matches for typos
        suggestions = difflib.get_close_matches(model_name, available_models)
        raise ValueError(f"Model '{model_name}' not found. Did you mean: {suggestions}?")

    model_class = getattr(models, model_name)

    # Load pre-trained model if available
    model = model_class(pretrained=True)

    # Adjust for number of classes (for CIFAR10, typically 10)
    if model_name.startswith('resnet') or model_name.startswith('densenet'):
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name.startswith('vgg') or model_name.startswith('alexnet'):
        model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, num_classes)

    return model


# 5. Training: Training Loop with Optional Early Stopping and Verbosity Control
Here’s how to set up the training loop, including a verbosity flag and early stopping:

In [ ]:
# Training loop with optional verbosity and early stopping
def train_model(model, train_loader, test_loader, epochs, learning_rate, patience=None, min_delta=0.01, verbosity=True):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    best_loss = np.Inf
    no_improvement = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        # Using tqdm for progress bar
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', disable=not verbosity)

        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation phase
        val_loss, val_acc = evaluate_model(model, test_loader, criterion, device)

        if verbosity:
            print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        # Early stopping logic
        if patience:
            if val_loss < best_loss - min_delta:
                best_loss = val_loss
                no_improvement = 0
            else:
                no_improvement += 1
                if no_improvement >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

# Function to evaluate model on test data
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_loss = running_loss / len(test_loader)
    test_acc = correct / total
    return test_loss, test_acc


# 6. Evaluation: Summary of Results
# Conclusion
This notebook demonstrated a flexible way to train and evaluate models on various datasets using PyTorch. It incorporates robust error handling, early stopping, and customizable verbosity for the training process.
